In [1]:
import pandas as pd
import numpy as np
import re
import difflib
import string
from xpinyin import Pinyin
py = Pinyin()
pd.set_option('display.max_rows', None)
p = re.compile(r'[（](.*?)[）]', re.S)
def string_similar(s1, s2):#对比字符串相似度
    return difflib.SequenceMatcher(None, s1, s2).quick_ratio()

In [2]:
fpath = './data.xlsx'
pvuv = pd.read_excel(fpath)

In [3]:
df=pvuv[['authors','organization']].loc[1:8932]#loc 切片

In [4]:
def process(data):
    #删除\u3000
    data["authors"] = data["authors"].replace('\u3000','')
    #删除１
    data["authors"] = data["authors"].replace('１','')
    #删除
    data["authors"] = data["authors"].replace('','')
    #删除括号
    data["authors"] = re.sub(p,'',data["authors"])
    #删除括号右边的内容
    data["authors"] = data["authors"].split('(')[0]
    #删除[]
    data["authors"] = data["authors"].replace('[','').replace(']','') 
    #删除国家
    data["authors"] = data["authors"].replace('zh_CN','')
    #删除数字
    data["authors"] = re.sub('[\d]','',data["authors"])
    #将:改为;
    data["authors"] = data["authors"].replace(':',';')
    
    return data

In [5]:
df.loc[:,"authors"]=df.apply(process,axis=1)

2782：'Boxuan Yue, Qinghua Chi, Kangling Liu,;Jun Liang'
2878：'Zhou;J.L.;Wang;X.;Zhang;J.F.;Wang;H.;Yang;G.H.'

In [6]:
def cn(data):#判断中文
    res = True
    for w in data.replace(' ','').replace(',','').replace('，','').replace(';','').replace('；','').replace('.','').replace('·',''):
        if not '\u4e00' <= w <= '\u9fff':
            res = False
            break
    return res

In [7]:
def fuhao(data):
    return data.replace(' ','').replace(',','').replace('，','').replace(';','').replace('；','').replace('.','').replace('·','').replace('-','')

In [8]:
def name(data):
    result1 = py.get_pinyin(data).upper()
    s = result1.split('-')
    s1 = s[0]
    del s[0]
    result2=s1+''.join(i[0] for i in s)
    return result2

In [9]:
def judge(str1,data):#判断可不可以替代
    global authors_list
    res=False
    if len(authors_list)>0:
        if cn(str1):#如果这次操作的是一个中文
            for i in range(len(authors_list)):
                str2 = authors_list.loc[i]["authors"]
                if cn(str2):#str1和str2都是中文
                    if str1==str2:
                        if (data["organization"]==authors_list.loc[i,"organization"]):
                            res = True
                            break
                else:
                    str1_py = py.get_pinyin(str1).upper().replace('-','')#全名拼音
                    str1_py_s = name(str1)#拼音名缩写
                    if (string_similar(fuhao(str1_py), fuhao(str2))>0.89) or (string_similar(fuhao(str1_py_s), fuhao(str2))>=0.8):
                        if (data["organization"]==authors_list.loc[i,"organization"]):
                            res = True
                            break
        else:#str1是一个英文
            str1_py = py.get_pinyin(str1).upper().replace('-','')#全名拼音
            str1_py_s = name(str1)#拼音名缩写
            for i in range(len(authors_list)):
                str2=authors_list.loc[i]["authors"]
                str2_py = py.get_pinyin(str2).upper().replace('-','')#全名拼音
                str2_py_s = name(str2)
                if cn(str2):#str2是中文:
                    if (string_similar(fuhao(str2_py), fuhao(str1_py))>0.89) or (string_similar(fuhao(str2_py_s), fuhao(str1_py))>=0.8):
                        if (data["organization"]==authors_list.loc[i,"organization"]):
                            res = True
                            break
                else:#两个都是英文
                    if (string_similar(fuhao(str2_py),fuhao(str1_py))>0.89):
                        if (data["organization"]==authors_list.loc[i,"organization"]):
                            res = True
                            break
    return res

In [10]:
def add_list(data,lista):
    global authors_list
    while ' ' in lista:
        lista.remove(' ')
    while '' in lista:
        lista.remove('')
    for i in range(len(lista)):
        if not (judge(lista[i],data)):
            authors_list=authors_list.append({"authors":lista[i],"organization":data["organization"]},ignore_index=True)

In [11]:
def chinese0_pro(data):
    lista=data["authors"].split(' ')
    if '等' in lista:
        del lista[-1]
    for i in range(len(lista)):
        if i < len(lista):
            if len(lista[i])==1:
                if len(lista[i+1])==1:
                    lista[i]+=lista[i+1]
                    del lista[i+1]
    add_list(data,lista)

In [12]:
def chinese1_pro(data):
    lista=data["authors"].replace(' ','').replace('；',';').replace(',',';').replace('，',';').split(';')
    add_list(data,lista)

In [13]:
def chinese2_pro(data):
    lista=data["authors"].replace('；',';').replace(',',';').replace('，',';').split(';')
    add_list(data,lista)

In [14]:
def english0and1_pro(data):
    lista=data["authors"].split(';')
    add_list(data,lista)

In [15]:
def english0_1_pro(data):
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=5)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1   
    add_list(data,lista)

In [16]:
def english2_pro(data):#只包含空格和逗号的英文
    lista=data["authors"].split('，')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=8)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    add_list(data,lista)

In [17]:
def english3_pro(data):#只包括空格-;
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=5)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    add_list(data,lista)

In [18]:
def english4_pro(data):
    lista=data["authors"].split(';')
    add_list(data,lista)

2782：'Boxuan Yue, Qinghua Chi, Kangling Liu,;Jun Liang'
2878：'Zhou;J.L.;Wang;X.;Zhang;J.F.;Wang;H.;Yang;G.H.'
2879：Zhou;J. L.;Wang X;Zhang J.F. and Wang H.
6480:'Lijun Liu, Shuming Cheng, Bo Qi, Zairong Xi,;Daoyi Dong'

In [19]:
def english5_pro(data):
    lista=data["authors"].replace('.;',';').replace('.,',';').replace('.，',';').replace('and',';').split(';')
    if 'Pand' in data["authors"]:
        lista=data["authors"].replace('.;',';').replace('.,',';').replace('.，',';').split(';')
    if data["authors"] == df.loc[2782]["authors"] or data["authors"] == df.loc[6480]["authors"]:
        lista=data["authors"].replace(',;',',').split(',')
    elif data["authors"] == df.loc[2878]["authors"]:
        lista=data["authors"].split('.;')
    elif data["authors"] == df.loc[2879]["authors"]:
        lista=['Zhou;J. L.','Wang X','Zhang J.F.','Wang H.']
    while '' in lista:
        lista.remove('')
    add_list(data,lista)

In [20]:
def english6_pro(data):
    lista=data["authors"].replace('and',';').split(';')
    if 'Pand' in data["authors"] or 'uand' in data["authors"]:
        lista=data["authors"].split(';')
    if ';' not in data["authors"]:
        if len(data["authors"])<11:
            lista=data["authors"].split(';')
        else:
            lista=data["authors"].split(',')
    if data["authors"] == df.loc[7132]["authors"]:
        lista=data["authors"].replace(',',';').split(';')
    if data["authors"] == df.loc[6241]["authors"]:
        lista=data["authors"].split(';')
    add_list(data,lista)

In [21]:
def english7_pro(data):
    lista=data["authors"].split(';')
    add_list(data,lista)

In [22]:
def english8_pro(data):
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<5)
        if '.' in lista[ak]:
            a == False 
        if a==True:#存在小于五个的情况
            
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    add_list(data,lista)

In [23]:
def english9_pro(data):
    lista=data["authors"].replace('and',',').split(',')
    add_list(data,lista)

In [24]:
def english10_pro(data):
    lista=data["authors"].split('，')
    add_list(data,lista)

In [25]:
def english11_pro(data):
    lista=data["authors"].split('，')
    add_list(data,lista)

In [26]:
def english12_pro(data):
    lista=data["authors"].split(',')
    add_list(data,lista)

In [27]:
def english13_pro(data):
    lista=data["authors"].split(';')
    add_list(data,lista)

In [28]:
def english14_pro(data):
    lista=data["authors"].split('，')
    add_list(data,lista)

In [29]:
def english15_pro(data):
    lista=data["authors"].split('，')
    add_list(data,lista)

In [30]:
def lang0_0_pro(data):
    lista=data["authors"].replace('；',';').split(';')
    if data["authors"] == '赵春晖 Wei Wang':
        lista=['赵春晖', 'Wei Wang']
    if data["authors"] == '祝乔;Yin;Yi-Xin;Cheng;Han-Qing;Chen;Xian-Zhong':
        lista=['祝乔','Yin Yi-Xin','Cheng Han-Qing','Chen;Xian-Zhong']
    add_list(data,lista)

In [31]:
def lang0_1_pro(data):
    lista=data["authors"].split('，')
    add_list(data,lista)

In [32]:
def lang0_2_pro(data):
    lista=data["authors"].replace(',',';').split(';')
    add_list(data,lista)

In [33]:
def lang0_pro(data):
    lista=data["authors"].replace('第一通讯作者','').split(';')
    add_list(data,lista)

In [34]:
def lang1_pro(data):
    lista=data["authors"].replace('li&','').split(';')
    if data["authors"] == 'Cheng;Y、Mao;JF、Wu;AH 、 Wu;GQ;Mao Jingfeng;Wu Aihua;Wu Guoqing':
        lista=['ChengY','MaoJF','WuAH','WuGQ','Mao Jingfeng','Wu Aihua','Wu Guoqing']
    add_list(data,lista)

In [35]:
def lang2_pro(data):
    if data["authors"] == df.loc[4175,"authors"]:
        lista=['Yajun Zhang','Y. Jia','T. Chai','D. Wang','W. Dai','J. Fu']
    if data["authors"] == df.loc[4176,"authors"]:
        lista=['Yajun Zhang','T. Chai','H. Wang','D. Wang','X. Chen']
    if data["authors"] == df.loc[4556,"authors"]:
        lista=['Xiaokun Dai','Yang Song','Mira Schüller','Dieter Schramm']
    if data["authors"] == df.loc[4612,"authors"]:
        lista=['Xianzhen Zhang','Zuohua Liu','Hui Peng','Xianmin Zhan']
    if data["authors"] == df.loc[4617,"authors"]:
        lista=['Xianmin Zhang','Praveen Agarwal','Zuohua Liu','Hui Peng']
    if (data["authors"] == df.loc[5924,"authors"]) or (data["authors"] == df.loc[5925,"authors"]):
        lista=['Qi Lü']
    if data["authors"] == df.loc[6808,"authors"]:
        lista=['Kwon,Joseph Sang-Il','Nayhouse,Michael','Orkoulas,Gerassimos','Ni, Dong','Christofides, Panagiotis D.']
    if data["authors"] == df.loc[8366,"authors"]:
        lista=['Crose, Marquis','Kwon, Joseph Sang-Il','Nayhouse, Michael','Ni, Dong','Christofides, Panagiotis D.']    
    add_list(data,lista)

In [36]:
def authors_pro(data):
    if data["authors_type"]=="chinese0":#只包括空格的中文
        chinese0_pro(data)
    elif data["authors_type"]=="chinese1":#只包含符号;；,·，的中文
        chinese1_pro(data)
    elif data["authors_type"]=="chinese2":#只包含符号;；,·，.的中文
        chinese2_pro(data)
    elif data["authors_type"]=="english0":#只包括空格的英文
        english0and1_pro(data)
    elif data["authors_type"]=="english0_1":#只包括空格和分号的英文
        english0_1_pro(data)
    elif data["authors_type"]=="english1":
        english0and1_pro(data)
    elif data["authors_type"]=="english2":
        english2_pro(data)
    elif data["authors_type"]=="english3":
        english3_pro(data)
    elif data["authors_type"]=="english4":
        english4_pro(data)
    elif data["authors_type"]=="english5":
        english5_pro(data)
    elif data["authors_type"]=="english6":
        english6_pro(data)
    elif data["authors_type"]=="english7":
        english7_pro(data)
    elif data["authors_type"]=="english8":
        english8_pro(data)
    elif data["authors_type"]=="english9":
        english9_pro(data)
    elif data["authors_type"]=="english10":
        english10_pro(data)
    elif data["authors_type"]=="english11":
        english11_pro(data)
    elif data["authors_type"]=="english12":
        english12_pro(data)
    elif data["authors_type"]=="english13":
        english13_pro(data)
    elif data["authors_type"]=="english14":
        english14_pro(data)
    elif data["authors_type"]=="english15":
        english15_pro(data)
    elif data["authors_type"]=="lang0_0":
        lang0_0_pro(data)
    elif data["authors_type"]=="lang0_1":
        lang0_1_pro(data)
    elif data["authors_type"]=="lang0_2":
        lang0_2_pro(data)
    elif data["authors_type"]=="lang0":
        lang0_pro(data)
    elif data["authors_type"]=="lang1":
        lang1_pro(data)
    elif data["authors_type"]=="lang2":
        lang2_pro(data)

In [37]:
for i in range(len(df)):
    authors_pro(df.loc[i+1])

KeyError: 'authors_type'

############################################################################
############################################################################
############################################################################
列表整理完毕 接下来根据列表作出关系列表

In [38]:
authors_cp=pd.DataFrame(columns=["author_a","author_b","organization"])

In [173]:
stra=''
strb=''

In [174]:
authors_list.loc[1,"authors"]#.loc[df["organization"]=="南通大学"]

'丛爽'

In [39]:
def cppd(str1,str2):
    if cn(str1):#如果这次操作的是一个中文
        if cn(str2):#str1和str2都是中文
            if (str1.replace(' ','').replace('.',''))==(str2.replace(' ','').replace('.','')):
                return True
            else:
                return False
        else:
            str1_py = py.get_pinyin(str1).upper().replace('-','')#全名拼音
            str1_py_s = name(str1)#拼音名缩写
            if (string_similar(fuhao(str1_py), fuhao(str2))>0.89) or (string_similar(fuhao(str1_py_s), fuhao(str2))>=0.8):
                return True
            else:
                return False
    else:#str1是一个英文
        str1_py = py.get_pinyin(str1).upper().replace('-','')#全名拼音
        str1_py_s = name(str1)#拼音名缩写
        str2_py = py.get_pinyin(str2).upper().replace('-','')#全名拼音
        str2_py_s = name(str2)
        if cn(str2):#str2是中文:
            if (string_similar(fuhao(str2_py), fuhao(str1_py))>0.89) or (string_similar(fuhao(str2_py_s), fuhao(str1_py))>=0.8):
                return True
            else:
                return False
        else:#两个都是英文
            if (string_similar(fuhao(str2_py),fuhao(str1_py))>0.89):
                return True
            else:
                return False

In [176]:
authors_list.loc[1,'organization']

'中国科学技术大学'

In [40]:
def xunzhaolist(str1,school):#输入一个名字，返回其在list中的名字
    for i in range(len(authors_list)):
        if cppd(str1,authors_list.loc[i,'authors']) and authors_list.loc[i,'organization']==school:
            return authors_list.loc[i,'authors']

In [41]:
def cp_judge(str1,str2,cp_list,school):#判断可不可以替代
    global authors_cp
    global stra
    global strb
    res=False #res表示cp_list中有无现在的组合 False为无，True为有
    if len(authors_cp)>0:
        for i in range(len(authors_cp)):
            if (school!=authors_cp.loc[i,"organization"]):
                continue
            else:
                str_1=cp_list.loc[i]["author_a"]#str_1是关系列表中的第一个人
                str_2=cp_list.loc[i]["author_b"]#str_2是关系列表中的第二个人
                str1=xunzhaolist(str1,school)
                str2=xunzhaolist(str2,school)
                stra=str1
                strb=str2
                if ((cppd(str1,str_1) and cppd(str2,str_2)) or (cppd(str2,str_1) and cppd(str1,str_2))):#如果是真 说明可以替换 直接掠过
                    res = True
    return res

In [42]:
def cp_add_list(data,lista):
    global authors_list
    global authors_cp
    while ' ' in lista:
        lista.remove(' ')
    while '' in lista:
        lista.remove('')
    if len(lista)>1:
        for i in range(len(lista)):
            j=(len(lista)-1)
            a=(j>i)
            if a:
                if not cp_judge(lista[i],lista[j],authors_cp,data["organization"]):
                    #print(f'list:{lista},中的i:{i} j{j} 通过了')
                    authors_cp=authors_cp.append({"author_a":stra,"author_b":strb,"organization":data["organization"]},ignore_index=True)
            j-=1

In [43]:
def chinese0_pro_add(data):
    lista=data["authors"].split(' ')
    if '等' in lista:
        del lista[-1]
    for i in range(len(lista)):
        if i < len(lista):
            if len(lista[i])==1:
                if len(lista[i+1])==1:
                    lista[i]+=lista[i+1]
                    del lista[i+1]
    cp_add_list(data,lista)

In [44]:
def chinese1_pro_add(data):
    lista=data["authors"].replace(' ','').replace('；',';').replace(',',';').replace('，',';').split(';')
    cp_add_list(data,lista)

In [45]:
def chinese2_pro_add(data):
    lista=data["authors"].replace('；',';').replace(',',';').replace('，',';').split(';')
    cp_add_list(data,lista)

In [46]:
def english0_1_pro_add(data):
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=5)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1   
    cp_add_list(data,lista)

In [47]:
def english2_pro_add(data):#只包含空格和逗号的英文
    lista=data["authors"].split('，')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=8)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    cp_add_list(data,lista)

In [48]:
def english3_pro_add(data):#只包括空格-;
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<=5)
        if a==True:#存在小于五个的情况
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    cp_add_list(data,lista)

In [49]:
def english4_pro_add(data):
    lista=data["authors"].split(';')
    cp_add_list(data,lista)

2782：'Boxuan Yue, Qinghua Chi, Kangling Liu,;Jun Liang'
2878：'Zhou;J.L.;Wang;X.;Zhang;J.F.;Wang;H.;Yang;G.H.'
2879：Zhou;J. L.;Wang X;Zhang J.F. and Wang H.
6480:'Lijun Liu, Shuming Cheng, Bo Qi, Zairong Xi,;Daoyi Dong'

In [50]:
def english5_pro_add(data):
    lista=data["authors"].replace('.;',';').replace('.,',';').replace('.，',';').replace('and',';').split(';')
    if 'Pand' in data["authors"]:
        lista=data["authors"].replace('.;',';').replace('.,',';').replace('.，',';').split(';')
    if data["authors"] == df.loc[2782]["authors"] or data["authors"] == df.loc[6480]["authors"]:
        lista=data["authors"].replace(',;',',').split(',')
    elif data["authors"] == df.loc[2878]["authors"]:
        lista=data["authors"].split('.;')
    elif data["authors"] == df.loc[2879]["authors"]:
        lista=['Zhou;J. L.','Wang X','Zhang J.F.','Wang H.']
    while '' in lista:
        lista.remove('')
    cp_add_list(data,lista)

In [51]:
def english6_pro_add(data):
    lista=data["authors"].replace('and',';').split(';')
    if 'Pand' in data["authors"] or 'uand' in data["authors"]:
        lista=data["authors"].split(';')
    if ';' not in data["authors"]:
        if len(data["authors"])<11:
            lista=data["authors"].split(';')
        else:
            lista=data["authors"].split(',')
    if data["authors"] == df.loc[7132]["authors"]:
        lista=data["authors"].replace(',',';').split(';')
    if data["authors"] == df.loc[6241]["authors"]:
        lista=data["authors"].split(';')
    cp_add_list(data,lista)

In [52]:
def english7_pro_add(data):
    lista=data["authors"].split(';')
    cp_add_list(data,lista)

In [53]:
def english8_pro_add(data):
    lista=data["authors"].split(';')
    listb=[]
    a=False
    ak = 0
    while '' in lista:
        lista.remove('')
    while ak<len(lista):
        a=(len(lista[ak])<5)
        if '.' in lista[ak]:
            a == False 
        if a==True:#存在小于五个的情况
            
            for i in range((int)(len(lista)/2)):
                listb.append(lista[2*i-2]+lista[2*i-1]) 
                lista=listb
                break
        ak+=1
    cp_add_list(data,lista)

In [54]:
def english9_pro_add(data):
    lista=data["authors"].replace('and',',').split(',')
    cp_add_list(data,lista)

In [55]:
def english10_pro_add(data):
    lista=data["authors"].split('，')
    cp_add_list(data,lista)

In [56]:
def english11_pro_add(data):
    lista=data["authors"].split('，')
    cp_add_list(data,lista)

In [57]:
def english12_pro_add(data):
    lista=data["authors"].split(',')
    cp_add_list(data,lista)

In [58]:
def english13_pro_add(data):
    lista=data["authors"].split(';')
    cp_add_list(data,lista)

In [59]:
def english14_pro_add(data):
    lista=data["authors"].split('，')
    cp_add_list(data,lista)

In [60]:
def english15_pro_add(data):
    lista=data["authors"].split('，')
    cp_add_list(data,lista)

In [61]:
def lang0_0_pro_add(data):
    lista=data["authors"].replace('；',';').split(';')
    if data["authors"] == '赵春晖 Wei Wang':
        lista=['赵春晖', 'Wei Wang']
    if data["authors"] == '祝乔;Yin;Yi-Xin;Cheng;Han-Qing;Chen;Xian-Zhong':
        lista=['祝乔','Yin Yi-Xin','Cheng Han-Qing','Chen;Xian-Zhong']
    cp_add_list(data,lista)

In [62]:
def lang0_1_pro_add(data):
    lista=data["authors"].split('，')
    cp_add_list(data,lista)

In [63]:
def lang0_2_pro_add(data):
    lista=data["authors"].replace(',',';').split(';')
    cp_add_list(data,lista)

In [64]:
def lang0_pro_add(data):
    lista=data["authors"].replace('第一通讯作者','').split(';')
    cp_add_list(data,lista)

In [65]:
def lang1_pro_add(data):
    lista=data["authors"].replace('li&','').split(';')
    if data["authors"] == 'Cheng;Y、Mao;JF、Wu;AH 、 Wu;GQ;Mao Jingfeng;Wu Aihua;Wu Guoqing':
        lista=['ChengY','MaoJF','WuAH','WuGQ','Mao Jingfeng','Wu Aihua','Wu Guoqing']
    cp_add_list(data,lista)

In [66]:
def lang2_pro_add(data):
    if data["authors"] == df.loc[4175,"authors"]:
        lista=['Yajun Zhang','Y. Jia','T. Chai','D. Wang','W. Dai','J. Fu']
    if data["authors"] == df.loc[4176,"authors"]:
        lista=['Yajun Zhang','T. Chai','H. Wang','D. Wang','X. Chen']
    if data["authors"] == df.loc[4556,"authors"]:
        lista=['Xiaokun Dai','Yang Song','Mira Schüller','Dieter Schramm']
    if data["authors"] == df.loc[4612,"authors"]:
        lista=['Xianzhen Zhang','Zuohua Liu','Hui Peng','Xianmin Zhan']
    if data["authors"] == df.loc[4617,"authors"]:
        lista=['Xianmin Zhang','Praveen Agarwal','Zuohua Liu','Hui Peng']
    if (data["authors"] == df.loc[5924,"authors"]) or (data["authors"] == df.loc[5925,"authors"]):
        lista=['Qi Lü']
    if data["authors"] == df.loc[6808,"authors"]:
        lista=['Kwon,Joseph Sang-Il','Nayhouse,Michael','Orkoulas,Gerassimos','Ni, Dong','Christofides, Panagiotis D.']
    if data["authors"] == df.loc[8366,"authors"]:
        lista=['Crose, Marquis','Kwon, Joseph Sang-Il','Nayhouse, Michael','Ni, Dong','Christofides, Panagiotis D.']    
    add_list(data,lista)

In [67]:
def english0and1_pro_add(data):
    lista=data["authors"].split(';')
    cp_add_list(data,lista)

In [68]:
def authors_cp_list(data):
    #if data["authors_type"]=="chinese0":#只包括空格的中文
    #    chinese0_pro_add(data)
    if data["authors_type"]=="chinese1":#只包含符号;；,·，的中文
        chinese1_pro_add(data)
    elif data["authors_type"]=="chinese2":#只包含符号;；,·，.的中文
        chinese2_pro_add(data)
    elif data["authors_type"]=="english0":#只包括空格的英文
        english0and1_pro_add(data)
    elif data["authors_type"]=="english0_1":#只包括空格和分号的英文
        english0_1_pro_add(data)
    elif data["authors_type"]=="english1":
        english0and1_pro_add(data)
    elif data["authors_type"]=="english2":
        english2_pro_add(data)
    elif data["authors_type"]=="english3":
        english3_pro_add(data)
    elif data["authors_type"]=="english4":
        english4_pro_add(data)
    elif data["authors_type"]=="english5":
        english5_pro_add(data)
    elif data["authors_type"]=="english6":
        english6_pro_add(data)
    elif data["authors_type"]=="english7":
        english7_pro_add(data)
    elif data["authors_type"]=="english8":
        english8_pro_add(data)
    elif data["authors_type"]=="english9":
        english9_pro_add(data)
    elif data["authors_type"]=="english10":
        english10_pro_add(data)
    elif data["authors_type"]=="english11":
        english11_pro_add(data)
    elif data["authors_type"]=="english12":
        english12_pro_add(data)
    elif data["authors_type"]=="english13":
        english13_pro_add(data)
    elif data["authors_type"]=="english14":
        english14_pro_add(data)
    elif data["authors_type"]=="english15":
        english15_pro_add(data)
    elif data["authors_type"]=="lang0_0":
        lang0_0_pro_add(data)
    elif data["authors_type"]=="lang0_1":
        lang0_1_pro_add(data)
    elif data["authors_type"]=="lang0_2":
        lang0_2_pro_add(data)
    elif data["authors_type"]=="lang0":
        lang0_pro_add(data)
    elif data["authors_type"]=="lang1":
        lang1_pro_add(data)
    elif data["authors_type"]=="lang2":
        lang2_pro_add(data)

In [69]:
authors_cp=pd.DataFrame(columns=["author_a","author_b","organization"])

In [70]:
for i in range(len(df)):
    print(i)
    authors_cp_list(df.loc[i+1])

0


KeyError: 'authors_type'

In [71]:
authors_cp

,author_a,author_b,organization


In [72]:
authors_cp.to_excel('作者关系.xlsx',header=True)

In [73]:
authors_list.to_excel('作者列表.xlsx',header=True)

NameError: name 'authors_list' is not defined

作者关系处理完毕
==============================================================================
================================================================================
=============================================================================

In [74]:
authors_cp=pd.read_excel('./作者关系.xlsx')

In [75]:
authors_list=pd.read_excel('./作者列表.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: './作者列表.xlsx'

In [76]:
authors_list.insert(loc=0,column="number",value='')

NameError: name 'authors_list' is not defined

In [77]:
authors_list.insert(loc=3,column="achievement",value=0)

NameError: name 'authors_list' is not defined

In [78]:
for i in range(len(authors_list)):
    authors_list.loc[[i],['number']]=str(i+1).rjust(4,'0')

NameError: name 'authors_list' is not defined

In [79]:
authorsnumber_cp=pd.DataFrame(columns=["author_a","author_b","organization"])#创建关系列表

In [ ]:
for i in range(len(authors_cp)):
    str1=authors_cp.loc[i,"author_a"]
    str2=authors_cp.loc[i,"author_b"]
    school=authors_cp.loc[i,'organization']
    number(str1,str2,school)